In [2]:
import geopandas as gpd
import pandas as pd

## Open State data

In [3]:
file_path = "data/cb_2018_us_county_500k/cb_2018_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)
counties_gdf = counties_gdf[counties_gdf["STATEFP"] != "09"][["geometry", "AFFGEOID"]]
counties_gdf = counties_gdf.to_crs(4326)

#### Format for CT

In [4]:
ct_gdf = gpd.read_file("data/CT_Planning_Regions.geojson")
ct_gdf["AFFGEOID"] = "0500000US" + ct_gdf["PlanningRegionFIPS_GEOID"]

In [8]:
counties_gdf = pd.concat(
    [ct_gdf[["geometry", "AFFGEOID"]], counties_gdf], ignore_index=True
)

## Get Mobile Home Data Percents

In [9]:
acs_mobile_home = pd.read_csv(
    "data/ACSDP5Y2023.DP04_2025-02-06T161358/ACSDP5Y2023.DP04-Data.csv",
    low_memory=False,
)

#### Get Column names

In [10]:
acs_column_names = pd.read_csv(
    "data/ACSDP5Y2023.DP04_2025-02-06T161358/ACSDP5Y2023.DP04-Column-Metadata.csv"
)

In [11]:
mobile_home_cols = list(
    acs_column_names[
        (acs_column_names["Column Name"] == "DP04_0014PE")
        | (acs_column_names["Column Name"] == "DP04_0014E")
    ]["Column Name"]
)

#### Back to Data

In [12]:
acs_mobile_home_data_per_county_df = acs_mobile_home[["GEO_ID", *mobile_home_cols]]
acs_mobile_home_data_per_county_df.columns = acs_mobile_home_data_per_county_df.iloc[0]
acs_mobile_home_data_per_county_df = acs_mobile_home_data_per_county_df[1:]
acs_mobile_home_data_per_county_df = acs_mobile_home_data_per_county_df.rename(
    columns={
        "Estimate!!UNITS IN STRUCTURE!!Total housing units!!Mobile home": "MOBILE_HOMES",
        "Percent!!UNITS IN STRUCTURE!!Total housing units!!Mobile home": "PERCENT",
        "Geography": "AFFGEOID",
    }
)

## Merge Data

In [23]:
acs_mobile_home_data_per_county_gdf = counties_gdf.merge(
    acs_mobile_home_data_per_county_df, on="AFFGEOID", how="left"
)

In [24]:
acs_mobile_home_data_per_county_df["county_id"] = acs_mobile_home_data_per_county_df[
    "AFFGEOID"
].str[9:]
acs_mobile_home_data_per_county_df["state_id"] = acs_mobile_home_data_per_county_df[
    "AFFGEOID"
].str[9:11]

In [25]:
acs_mobile_home_data_per_county_gdf = acs_mobile_home_data_per_county_gdf.fillna(0)
acs_mobile_home_data_per_county_gdf[
    "MOBILE_HOMES"
] = acs_mobile_home_data_per_county_gdf["MOBILE_HOMES"].astype(int)
acs_mobile_home_data_per_county_gdf["PERCENT"] = acs_mobile_home_data_per_county_gdf[
    "PERCENT"
].astype(float)

In [26]:
acs_mobile_home_data_per_county_gdf = acs_mobile_home_data_per_county_gdf.to_crs(9311)
acs_mobile_home_data_per_county_gdf.to_file("data/mobile_home_county.gpkg")

In [27]:
acs_mobile_home_data_per_county_gdf

,geometry,AFFGEOID,MOBILE_HOMES,PERCENT,county_id,state_id
0,"POLYGON ((2249596.518 22977.704, 2249609.32 22...",0500000US09110,2406,0.6,09110,09
1,"MULTIPOLYGON (((2201518.594 -58325.515, 220152...",0500000US09170,1118,0.5,09170,09
2,"MULTIPOLYGON (((2198361.226 -62558.943, 219822...",0500000US09120,253,0.2,09120,09
3,"POLYGON ((2275297.417 21684.436, 2275280.685 2...",0500000US09150,607,1.5,09150,09
4,"POLYGON ((2200443.773 -37712.741, 2200476.075 ...",0500000US09140,1531,0.8,09140,09
...,...,...,...,...,...,...
3229,"POLYGON ((-8018.633 -507058.613, -7534.089 -50...",0500000US31073,93,7.7,31073,31
3230,"POLYGON ((1486193.517 -320378.86, 1488440.469 ...",0500000US39075,906,6.2,39075,39
3231,"POLYGON ((67253.903 -1607796.404, 69345.064 -1...",0500000US48171,1714,12.0,48171,48
3232,"POLYGON ((968181.218 -166268.173, 968091.993 -...",0500000US55079,2158,0.5,55079,55
